# Data analysis for the ISMAR article

In [53]:
# See: http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-matplotlib
%matplotlib inline

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from ast import literal_eval
from os import listdirt
from os.path import join

In [64]:
# Load the csv data files

data_path = "data/"

participants = pd.read_csv(join(data_path, "participants.csv"))

trials = []
for file in listdir(data_path):
    file_path = join(data_path, file)
    if file.endswith("trials.csv"):
        participant_trials = pd.read_csv(file_path)
        trials.append(participant_trials)

# Uncoment to visualize the differents arrays
#participants
#logs[0]

[[9, 9, 9, 9, 9], [13, 13, 13, 13, 13], [14, 14, 14, 14, 14], [10, 10, 10, 10, 10], [8, 8, 8, 8, 8]]


,participantId,sex,rightHanded,test,grid_config
0,0,M,True,"(1,2,3)","[[[9, 9, 9, 9, 9], [13, 13, 13, 13, 13], [14, ..."
1,1,F,False,"(4,3,1)",NaN


In [1]:
# See to help with analysis: http://www.scipy-lectures.org/packages/statistics/index.html#statistics